# Test Suite for the online training software

Unit Tests focus on individual components in isolation, ensuring that each part works correctly by itself. The primary purpose of unit tests is to ensure that each individual part (or "unit") of the software works as intended. By isolating each part of the program, developers can pinpoint where issues occur and ensure that each part behaves correctly independently. The main characteristics of unit testing are the following:
 - **Isolation:** unit tests focus on a small, specific piece of functionality, typically a single function or method. They do not interact with external systems like databases or web services;
 - **Velocity:** because they test small pieces of code, unit tests are generally quick to execute;
 - **Automation:** they are often run automatically as part of a continuous integration process;
 - **Repeatability:** unit tests should produce the same results every time they run, assuming the code has not changed.


Instead, integration tests focus on the interaction between components, ensuring that combined parts of the system work together as expected.
The primary purpose of integration tests is to detect issues that occur when units or modules are combined. These issues might not be apparent when units are tested in isolation, such as problems with data formats, incorrect method calls, or issues with external systems. The main characteristics of unit testing are the following:
 - **Combined Testing:** integration tests check the interactions between multiple units, modules, or services;
 - **Complexity:** they often involve more complex scenarios than unit tests and might require setting up and tearing down parts of the system, such as databases or network services;
 - **Slower testing:** Because they test more extensive parts of the system, integration tests typically take longer to run than unit tests.
 - **Realistic:** They aim to test the system in conditions closer to the real-world usage, which may involve external dependencies.

Both types of tests are crucial for ensuring the quality and reliability of software, with unit tests providing a solid foundation and integration tests ensuring that the overall system functions correctly when components are combined.


# Imports
The following code block provides the module imported by the test suite.

In [53]:
from unittest import TestCase, main
import uuid
from enum import Enum

# Platform intefaces
The following classes implements the basic interfaces that the final platform must be compliant with in order to pass the unit and integration tests.

In [54]:
class UserProfile(Enum):
  Teacher = 'teacher'
  Student = 'student'


class GroupRegisty():
  GROUPS = {}

  @staticmethod
  def create_group(grp_name) -> str:
    grp_id = uuid.uuid5(namespace=uuid.NAMESPACE_DNS, name=grp_name)
    grp = Group(grp_id, grp_name)

    if grp_id not in GroupRegisty.GROUPS:
      GroupRegisty.GROUPS[grp_id] = grp
      return grp_id
    else:
      raise Exception(msg='A group with the given name and ID already exists.')

  @staticmethod
  def get_group(grp_id) -> 'Group':

    if grp_id in GroupRegisty.GROUPS:
      return GroupRegisty.GROUPS[grp_id]
    else:
      raise Exception(msg='No group with the given ID exists.')

  @staticmethod
  def delete_group(grp_id):
    if grp_id in GroupRegisty.GROUPS:
      del(GroupRegisty.GROUPS[grp_id])
      return True
    else:
      raise Exception(msg='A group with the given ID does not exist.')

  @staticmethod
  def update_group(grp_name: str, grp: 'Group'):
    grp_id = uuid.uuid5(namespace=uuid.NAMESPACE_DNS, name=grp_name)
    if grp_id in GroupRegisty.GROUPS:
      del(GroupRegisty.GROUPS[grp_id])
      return GroupRegisty.create_group(grp.name)
    else:
      raise Exception(msg='A group with the given ID does not exist.')

  @staticmethod
  def add_user(grp_id: str, usr_id: str, profile: UserProfile):
    usr: 'User' = UserRegisty.get_user(usr_id)
    grp: 'Group' = GroupRegisty.get_group(grp_id)

    return grp.add_user(usr, profile)

  @staticmethod
  def remove_user(grp_id: str, usr_id: str, profile: UserProfile):
    usr: 'User' = UserRegisty.get_user(usr_id)
    grp: 'Group' = GroupRegisty.get_group(grp_id)

    return grp.remove_user(usr_id, profile)

class UserRegisty():

  REGISTRY = {UserProfile.Teacher: {}, UserProfile.Student: {}}

  @staticmethod
  def create_user(profile: UserProfile, user_data: dict):
    if profile in UserRegisty.REGISTRY:
      _profile_r = UserRegisty.REGISTRY[profile]

      if 'email' in user_data:
        usr_id = uuid.uuid5(namespace=uuid.NAMESPACE_DNS, name=user_data['email'])

        if usr_id in _profile_r:
          raise Exception(msg='User already exists with the given ID.')
        else:
          _profile_r[usr_id] = User(usr_id, profile, user_data)

          return usr_id
      else:
        raise Exception(msg='No email provided')
    else:
      raise Exception(msg=f'No profile exists with the given name: {profile}')


  @staticmethod
  def delete_user(profile: UserProfile, usr_id: str):
    if profile in UserRegisty.REGISTRY:
      _profile_r = UserRegisty.REGISTRY[profile]

      if usr_id in _profile_r:
        del(_profile_r[usr_id])
        return True
      else:
        raise Exception(msg='No user exists with the given ID.')

    else:
      raise Exception(msg=f'No profile exists with the given name: {profile}')

  @staticmethod
  def update_user(profile: UserProfile, usr_id: str, user_data: dict):
    if profile in UserRegisty.REGISTRY:
      _profile_r = UserRegisty.REGISTRY[profile]

      if 'email' in user_data:
        new_usr_id = uuid.uuid5(namespace=uuid.NAMESPACE_DNS, name=user_data['email'])

        if usr_id in _profile_r:
          del(_profile_r[usr_id])
          _profile_r[new_usr_id] = User(new_usr_id, profile, user_data)
        else:
          raise Exception(msg='No user exists with the given ID.')
      else:
        raise Exception(msg='No email provided')
    else:
      raise Exception(msg=f'No profile exists with the given name: {profile}')

  @staticmethod
  def get_user(profile: UserProfile, usr_id: str):
    if profile in UserRegisty.REGISTRY:
      _profile_r = UserRegisty.REGISTRY[profile]

      if usr_id in _profile_r:
        return _profile_r[usr_id]
      else:
        raise Exception(msg=f'No user exists with the given usr_id: {usr_id}')
    else:
      raise Exception(msg=f'No profile exists with the given name: {profile}')


class User():

  def __init__(self, usr_id: str, profile: UserProfile, user_data: dict):
    self.__profile = profile
    self.__id = usr_id
    self.__user_data = user_data

  @property
  def profile(self) -> UserProfile:
    return self.__profile

  @property
  def id(self) -> str:
    return self.__id


  def __eq__(self, usr: 'User') -> bool:
    return self.id == usr.id

class Group():

  def __init__(self, grp_id: str, grp_name: str):
    self.__id = grp_id
    self.__name = grp_name
    self.__users = {UserProfile.Student: {}, UserProfile.Teacher: {}}

  @property
  def id(self):
    return self.__id

  @property
  def name(self):
    return self.name

  def add_user(self, usr: User, profile: UserProfile):

    if profile in self.users:

      users = self.users[profile]
      if usr.id in users:
        raise Exception(msg='User already exists with the given ID.')
      else:
        users[usr.id] = usr

        return True

    raise Exception(msg='The selected profile is not allowed as it does not exist.')

  def remove_user(self, usr_id: str, profile: UserProfile):

    if profile in self.users:

      users = self.users[profile]
      if usr_id not in users:
        raise Exception(msg='No user exists with the given ID.')
      else:
        del(users[usr_id])

    raise Exception(msg='The selected profile is not allowed as it does not exist.')

# Test Suite

The following is the definition of the tests that implement the test suite.

In [55]:
class FOSSRTestCase(TestCase):

  def setUp(self):
    usr_data = {'given_name': 'John', 'family_name': 'Donne', 'email': 'john.donne@fossr.eu'}

    self.usr_data = usr_data
    self.usr_id = UserRegisty.create_user(UserProfile.Student, usr_data)
    self.usr_id = UserRegisty.create_user(UserProfile.Teacher, usr_data)
    print('setup')

  def tearDown(self):
    try:
      UserRegisty.delete_user(UserProfile.Student, self.usr_id)
      UserRegisty.delete_user(UserProfile.Teacher, self.usr_id)
      self.usr_data = None
    except Exception as e:
      pass
    print('teardown')

  def test_user_cretion(self):

    print('User creation: FR-0101 - The system shall allow the creation of users with different profiles, i.e. teacher or student.')

    __usr_data = {'given_name': 'George', 'family_name': 'Herbert', 'email': 'george.herbert@fossr.eu'}
    usr_id: str = UserRegisty.create_user(UserProfile.Student, __usr_data)
    self.assertIsNotNone(usr_id)

    _usr_id = uuid.uuid5(namespace=uuid.NAMESPACE_DNS, name=__usr_data['email'])
    _user = User(usr_id, UserProfile.Student, __usr_data)

    user = UserRegisty.get_user(UserProfile.Student, usr_id)

    self.assertEqual(_user, user)

    print('\tSuccess creation of a user with Student profile')

    usr_id: str = UserRegisty.create_user(UserProfile.Teacher, __usr_data)
    self.assertIsNotNone(usr_id)

    _usr_id = uuid.uuid5(namespace=uuid.NAMESPACE_DNS, name=__usr_data['email'])
    _user = User(usr_id, UserProfile.Teacher, __usr_data)

    user = UserRegisty.get_user(UserProfile.Teacher, usr_id)

    self.assertEqual(_user, user)

    print('\tSuccess creation of a user with Teacher profile')

  def test_user_modification(self):

    print('User modification: FR-0104 - The system shall allow the modification of a user.')

    usr_id = uuid.uuid5(namespace=uuid.NAMESPACE_DNS, name=self.usr_data['email'])

    user = UserRegisty.get_user(UserProfile.Student, usr_id)

    # We change the email
    new_usr_data = {'given_name': 'John', 'family_name': 'Donne', 'email': 'j_donne@fossr.eu'}

    UserRegisty.update_user(UserProfile.Student, usr_id, new_usr_data)
    _usr_id = uuid.uuid5(namespace=uuid.NAMESPACE_DNS, name=new_usr_data['email'])

    _user = UserRegisty.get_user(UserProfile.Student, _usr_id)

    self.assertNotEqual(_user, user)

    print('\tSuccess modification of a user with Student profile')

    user = UserRegisty.get_user(UserProfile.Teacher, usr_id)

    # We change the email
    new_usr_data = {'given_name': 'John', 'family_name': 'Donne', 'email': 'j_donne@fossr.eu'}

    UserRegisty.update_user(UserProfile.Teacher, usr_id, new_usr_data)
    _usr_id = uuid.uuid5(namespace=uuid.NAMESPACE_DNS, name=new_usr_data['email'])

    _user = UserRegisty.get_user(UserProfile.Teacher, _usr_id)

    self.assertNotEqual(_user, user)

    print('\tSuccess modification of a user with Teacher profile')

  def test_user_deletion(self):
    print('User deletion: FR-0106 - The system shall allow the deletion of a user.')

    usr_id = uuid.uuid5(namespace=uuid.NAMESPACE_DNS, name=self.usr_data['email'])
    ret = UserRegisty.delete_user(UserProfile.Student, usr_id)
    self.assertTrue(ret)
    print('\tSuccess deletion of a user with Student profile')
    usr_id = uuid.uuid5(namespace=uuid.NAMESPACE_DNS, name=self.usr_data['email'])
    ret = UserRegisty.delete_user(UserProfile.Teacher, usr_id)
    self.assertTrue(ret)
    print('\tSuccess deletion of a user with Teacher profile')

  def test_group_creation(self):
    print('Group creation: FR-0102 - The system shall allow the creation of groups for different user profiles, i.e. teacher or student.')

    grp_id = GroupRegisty.create_group('FOSSR')
    self.assertIsNotNone(grp_id)

    print('\tSuccess creation of a group.')

  def test_group_deletion(self):
    print('Group deletion: FR-0105 - The system shall allow the deletion of a group.')

    grp_id = GroupRegisty.create_group('FOSSR project')
    ret = GroupRegisty.delete_group(grp_id)
    self.assertTrue(ret)

    print('\tSuccess deletion of a group.')




# Tests exution

The following code block allows the automation of the execution of the tests.

In [56]:
if __name__ == '__main__':

  print('*** START OF TEST CASES ***')
  print('')

  main(argv=['first-arg-is-ignored'], exit=False)

  print('')
  print('*** END OF TEST CASES ***')


.....
----------------------------------------------------------------------
Ran 5 tests in 0.013s

OK


*** START OF TEST CASES ***

setup
Group creation: FR-0102 - The system shall allow the creation of groups for different user profiles, i.e. teacher or student.
	Success creation of a group.
teardown
setup
Group deletion: FR-0105 - The system shall allow the deletion of a group.
	Success deletion of a group.
teardown
setup
User creation: FR-0101 - The system shall allow the creation of users with different profiles, i.e. teacher or student.
	Success creation of a user with Student profile
	Success creation of a user with Teacher profile
teardown
setup
User deletion: FR-0106 - The system shall allow the deletion of a user.
	Success deletion of a user with Student profile
	Success deletion of a user with Teacher profile
teardown
setup
User modification: FR-0104 - The system shall allow the modification of a user.
	Success modification of a user with Student profile
	Success modification of a user with Teacher profile
teardown

*** END OF TEST CASES ***
